In [ ]:
%pip install langchain
%pip install langchain-openai
%pip install langchain-community
%pip install langchainhub
%pip install pypdf
%pip install faiss-cpu

For this notebook besides the OPENAI API KEY, you will need the LANGCHAIN API KEY (if you want to use langsmith platform for evaluation) and the TAVILY API KEY. 

1. [See here for the quick setup for LANGSMITH](https://docs.smith.langchain.com/).
2. [See here for the quick setup for TAVILY API KEY](https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/).

In [46]:
import os
import getpass

# # Set OPENAI API Key

# os.environ["OPENAI_API_KEY"] = getpass.getpass()
# os.environ["TAVILY_API_KEY"] = getpass.getpass() # you can obtain your tavily api key here: https://tavily.com/
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass() # obtain your langsmith api key here: https://smith.langchain.com/settings
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_PROJECT="default"
# OR (load from .env file)
# make sure you have python-dotenv installed
# from dotenv import load_dotenv
# load_dotenv("./.env")

Create your api key in the settings page of the [langsmith platform here](https://smith.langchain.com/settings).

> Summarized from this awesome video about the discussion of chains and agents: https://www.youtube.com/watch?v=bYLHklxEd_k&t=1801s


- Chains and agents represent two concepts in technology with distinct functionalities. Chains refer to linear sequences of events or processes that rely on the step-by-step execution of tasks, often requiring predefined inputs and producing predictable outcomes. 

- Agents, conversely, are autonomous systems capable of making decisions and performing tasks independently, adapting to new situations without explicit instructions. While chains excel in structured environments with clear rules, agents are designed to navigate complex, dynamic scenarios, learning and adjusting their behavior over time. The core difference lies in the level of autonomy and complexity each can handle, with agents typically seen as more advanced due to their self-governing nature.

# The Modern Tool Calling Way of LangChain

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' times 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the 'y'."""
    return x**y

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

tools = [multiply, exponentiate, add]


llm = ChatOpenAI(model="gpt-4o", temperature=0)


agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241", })



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'x': 5, 'y': 2.743}`


82.65606421491815
Invoking: `add` with `{'x': 3, 'y': 0}`


3.0
Invoking: `add` with `{'x': 17.24, 'y': -918.1241}`


-900.8841
Invoking: `add` with `{'x': 3.0, 'y': 82.65606421491815}`


85.65606421491815The result of \(3 + 5^{2.743}\) is approximately 85.656, and \(17.24 - 918.1241\) is \(-900.8841\).

> Finished chain.


{'input': "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241",
 'output': 'The result of \\(3 + 5^{2.743}\\) is approximately 85.656, and \\(17.24 - 918.1241\\) is \\(-900.8841\\).'}

source: https://arc.net/l/quote/cdomnqvm

So when should you use with_structured_output versus binding tools and reading tool calls directly?

with_structured_output always returns a structured output in the schema that you specified. This is useful when you want to force the LLM to output information that matches a specific schema. This is useful for information extraction tasks.

bind_tools is more general and can select a specific tool - or no tool, or multiple tools! This is useful when you want to allow the LLM to have more flexibility in how it should respond - for example, in agent applications where you need to choose which tools to invoke but also respond to the user.

**Tool Calling Also with Pydantic**

In [ ]:
from typing import Optional

from pydantic import BaseModel, Field


# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )




llm_with_structured_output = llm.with_structured_output(Joke)

In [ ]:
llm_with_structured_output.invoke("Joke about how cats are evil")

Joke(setup='Why did the cat sit on the computer?', punchline='Because it wanted to keep an eye on the mouse!', rating=7)

# Tool Calling

In [2]:
from langchain_openai import ChatOpenAI
def send_error_message(message: str) -> str:
    """Send an error message to the user."""
    return f"Error: {message}"

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_tools = llm.bind_tools([send_error_message])
llm_tools.invoke("If the following request is impossible to achieve, send an error message to the user with a funny punchline:\
    My request is that I become the master of the universe with all the money and intelligence that surpasses any human ever.")
# Pydantic


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tai62d2YQcxe3zzUKRTys5dK', 'function': {'arguments': '{"message":"Becoming the master of the universe is a bit tricky—last I checked, the universe doesn\'t have a \'For Sale\' sign! But hey, you can always aim for the title of \'Master of Your Own Living Room\'!"}', 'name': 'send_error_message'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 93, 'total_tokens': 154, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9f9c6a05-ba38-4108-896e-f25c6b5d01b6-0', tool_calls=[{'name': 'send_error_message', 'args': {'message': "Becoming the master of the universe is a bit tricky—last I checked, the universe doesn't have a 'For Sa

In [ ]:
class send_error_message(BaseModel):
    """Send an error message to the user."""
    message: str = Field(description="The error message to send to the user")


llm_tools = llm.bind_tools([send_error_message])

In [ ]:
llm_tools.invoke("If the following request is impossible to achieve, send an error message to the user with a funny punchline:\
    My request is that I become the master of the universe with all the money and intelligence that surpasses any human ever.")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kvSPhUyMEbAPuK1yqjZcXt0o', 'function': {'arguments': '{"message":"Becoming the master of the universe is a tall order! But hey, if you find a genie, just make sure to read the fine print—last time someone wished for that, they ended up with a pet rock instead!"}', 'name': 'send_error_message'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 103, 'total_tokens': 163, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-51df74b6-9917-4a86-9bc9-51408ceea558-0', tool_calls=[{'name': 'send_error_message', 'args': {'message': 'Becoming the master of the universe is a tall order! But hey, if you find a genie, just make sure to rea

In [ ]:
from langchain_core.output_parsers import PydanticToolsParser


query = "If the following request is impossible to achieve, send an error message to the user with a funny punchline:\
    My request is that I become the master of the universe with all the money and intelligence that surpasses any human ever."

chain = llm_tools | PydanticToolsParser(tools=[send_error_message])
chain.invoke(query)

[send_error_message(message="Becoming the master of the universe is a tall order! But hey, if you find a genie, just make sure to read the fine print—last time I checked, they don't cover 'universal domination' in their warranty!")]

In [ ]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_o0CgqR3HyqPbdAYWMHU60CN4', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_ewko6DtJ09MAR8QQipt4KNvK', 'type': 'tool_call'}]


In [ ]:
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_o0CgqR3HyqPbdAYWMHU60CN4', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_ewko6DtJ09MAR8QQipt4KNvK', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 87, 'total_tokens': 137, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-47449abe-d16b-4ce8-ab20-ab959b47cd8d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_o0CgqR3HyqPbdAYWMHU60CN4', 'type': 'tool_call'}, {'name': 'add'

In [ ]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

llm_with_tools.invoke(messages)

AIMessage(content='The result of \\(3 \\times 12\\) is 36, and the result of \\(11 + 49\\) is 60.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 153, 'total_tokens': 184, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-205504b0-1dd3-433c-97bc-a4550b9a8705-0', usage_metadata={'input_tokens': 153, 'output_tokens': 31, 'total_tokens': 184, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

Making a reporting agent.

Let's build an LLM agent that can build simple reports for tasks performed just by reading through the code and files inside a folder + notes from Notion page or .txt file.

First, let's organize our thoughts, an agent will be composed of:

- Tool
- Base LLM
- Conditions for stoping and returning an output 

![](2023-10-27-19-28-10.png)

In [29]:
MODEL='gpt-4o-mini'
TEMP=0.0

In [28]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [29]:
search = TavilySearchResults()

In [30]:
search.invoke("what is the weather in Paris?")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Paris', 'region': 'Ile-de-France', 'country': 'France', 'lat': 48.8667, 'lon': 2.3333, 'tz_id': 'Europe/Paris', 'localtime_epoch': 1728934658, 'localtime': '2024-10-14 21:37'}, 'current': {'last_updated_epoch': 1728934200, 'last_updated': '2024-10-14 21:30', 'temp_c': 16.2, 'temp_f': 61.2, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 341, 'wind_dir': 'NNW', 'pressure_mb': 1017.0, 'pressure_in': 30.03, 'precip_mm': 0.01, 'precip_in': 0.0, 'humidity': 88, 'cloud': 100, 'feelslike_c': 16.2, 'feelslike_f': 61.2, 'windchill_c': 16.5, 'windchill_f': 61.7, 'heatindex_c': 16.5, 'heatindex_f': 61.7, 'dewpoint_c': 13.2, 'dewpoint_f': 55.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 2.6, 'gust_kph': 4.1}}"},
 {'url': 'https://world-weather.info/forecast/france/paris/october-2024/',
 

In [47]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://python.langchain.com/docs/get_started/introduction")
docs = loader.load()
docs

[Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentIntegrationsAPI ReferenceMoreContributingPeopleLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG ApplicationBuild a PDF ingestion and Question/Answering systemBuild a Retrieval Augmented Generation (RAG) AppVector stores and retrieversBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use 

In [48]:
docs[0]

Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentIntegrationsAPI ReferenceMoreContributingPeopleLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG ApplicationBuild a PDF ingestion and Question/Answering systemBuild a Retrieval Augmented Generation (RAG) AppVector stores and retrieversBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a

In [49]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

documents

[Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Introduction | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Skip to main contentIntegrationsAPI ReferenceMoreContributingPeopleLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG Appli

In [50]:
vector = FAISS.from_documents(documents, OpenAIEmbeddings())

In [51]:
retriever = vector.as_retriever()

In [52]:
retriever.invoke("What is langchain?")[0]

Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='LangChain is a framework for developing applications powered by large language models (LLMs).\nLangChain simplifies every stage of the LLM application lifecycle:')

In [53]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langchain_search",
    "Search for information about LangChain. For any questions about LangChain, you must use this tool!",
)

In [54]:
tools = [search, retriever_tool]

In [55]:
from langchain_openai import ChatOpenAI

MODEL="gpt-4o-mini"
llm = ChatOpenAI(model=MODEL, temperature=0)

llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-10ad9bfc-6f59-40a1-b0ea-8d8aad77002b-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [56]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [57]:
prompt.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={})

In [58]:
prompt.messages[1]

MessagesPlaceholder(variable_name='chat_history', optional=True)

In [59]:
prompt.messages[2]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})

In [60]:
prompt.messages[3]

MessagesPlaceholder(variable_name='agent_scratchpad')

In [61]:
tools

[TavilySearchResults(api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))),
 Tool(name='langchain_search', description='Search for information about LangChain. For any questions about LangChain, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10d3ffe20>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12937bc90>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x10d8d2980>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12937bc90>, search_kwargs={}), document_prompt=PromptTemplate(input_va

In [62]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [63]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMes

In [64]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [65]:
agent_executor.invoke({"input": "What are some useful applications you can build with LangChain? List 5 in bullet points."})



> Entering new AgentExecutor chain...

Invoking: `langchain_search` with `{'query': 'useful applications of LangChain'}`


LangChain is a framework for developing applications powered by large language models (LLMs).
LangChain simplifies every stage of the LLM application lifecycle:

Integrations​
LangChain is part of a rich ecosystem of tools that integrate with our framework and build on top of it. Check out our growing list of integrations.
Contributing​
Check out the developer's guide for guidelines on contributing and help getting your dev environment set up.Edit this pageWas this page helpful?NextTutorialsTutorialsHow-to guidesConceptual guideAPI referenceEcosystem🦜🛠️ LangSmith🦜🕸️ LangGraphAdditional resourcesVersionsSecurityIntegrationsContributingCommunityTwitterGitHubOrganizationPythonJS/TSMoreHomepageBlogYouTubeCopyright © 2024 LangChain, Inc.

Development: Build your applications using LangChain's open-source building blocks, components, and third-party integrations.
Use L

{'input': 'What are some useful applications you can build with LangChain? List 5 in bullet points.',
 'output': "Here are five useful applications you can build with LangChain:\n\n- **Conversational Agents**: Create chatbots or virtual assistants that can engage in natural language conversations, providing customer support or information retrieval.\n\n- **Document Analysis**: Develop applications that can analyze and summarize large documents, extracting key information and insights for users.\n\n- **Personalized Recommendations**: Build systems that provide personalized content or product recommendations based on user preferences and behavior.\n\n- **Data Augmentation**: Use LangChain to generate synthetic data for training machine learning models, enhancing the diversity and volume of training datasets.\n\n- **Interactive Storytelling**: Create interactive narratives where users can influence the story's direction through their choices, powered by dynamic language generation."}

Now we can inspect what is going on in the [langsmith platform!](https://smith.langchain.com/o/b68c89e6-dae0-54ab-a84b-3201d39a3c50/settings).